In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
#from __future__ import print_function

## 2.1 Introduction

This chapter introduces more PyMC syntax and design patterns, and ways to think about how to model a system from a Bayesian perspective. It also contains tips and data visualization techniques for assesing goodness of fit for your Bayesian model.

#### 2.1.1 Parent and Child Relationships

To assist with describing Bayesisan relationships and to be consistent with PyMC's documentation, we introduce _parent_ and _child_ variables.

* **Parent variables** are variables that influence another variable.
* **Child variables** are variables that are affected by other variables - that is, are the subject of parent variables.

A variable can be both a parent and child. For example, consider the following PyMC code:

In [2]:
import pymc as pm

lambda_ = pm.Exponential("poisson_param", 1)
# used in the call to the next variable ...
data_generator = pm.Poisson("data_generator", lambda_)

data_plus_one = data_generator + 1

```lambda_``` controls the parameter of ```data_generator```, hence influences its values. The former is a parent of the latter. By symmetry, ```data_generator``` is a child of ```lambda_```.

Likewise, ```data_generator``` is a parent to the variable ```data_plus_one``` (hence making ```data_generator``` both a parent and child variable). Although it does not look like one, ```data_plus_one``` should be treated as a PyMC variable as it is a _function_ of another PyMC variable, hence is a child variable to ```data_generator```

This nomentclature is introduced to help us describe relationships in PyMC modeling. You can access a variable's children and parent variables using the ```children``` and ```parents``` attributes attached to variables.



In [3]:
print "Children of 'lambda_': "
print lambda_.children
print "\n Parents of 'data_generator': "
print data_generator.parents
print "\n Children of 'data_generator': "
print data_generator.children

Children of 'lambda_': 
set([<pymc.distributions.Poisson 'data_generator' at 0x000000000C7F2240>])

 Parents of 'data_generator': 
{'mu': <pymc.distributions.Exponential 'poisson_param' at 0x0000000008B965C0>}

 Children of 'data_generator': 
set([<pymc.PyMCObjects.Deterministic '(data_generator_add_1)' at 0x0000000008B964A8>])
